In [1]:
import face_recognition
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
original_mask = cv2.imread('CR.png')

In [5]:
process_this_frame = True


cap = cv2.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    save_fr = frame.copy()
    if process_this_frame:

        #smaller image for speed
        #small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        small_frame = frame
        

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_small_frame,model='cnn')
        face_landmarks_list = face_recognition.face_landmarks(rgb_small_frame,model='large',
                                                              face_locations = face_locations)
        
        
    process_this_frame = not process_this_frame #process every other imaage
    #cv2.addText(rgb_small_frame,)
    if len(face_landmarks_list) > 0:
        
        left_most = face_landmarks_list[0]['chin'][0][0]
        right_most = face_landmarks_list[0]['chin'][-1][0]
        width = abs(left_most-right_most)
        height = int((width/original_mask.shape[1])*original_mask.shape[0])
        mask = cv2.resize(original_mask,(width,height))
        how_high = face_landmarks_list[0]['left_eyebrow'][0][1] - face_landmarks_list[0]['nose_tip'][0][1]
        how_high = face_landmarks_list[0]['left_eyebrow'][0][1] + how_high
        
        if how_high < mask.shape[0]:
            mask = mask[mask.shape[0]-how_high:mask.shape[0] , 0 : mask.shape[1]]
            mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            _, masks_mask = cv2.threshold(mask_gray, 25, 255, cv2.THRESH_BINARY_INV)
            area = frame[0:mask.shape[0] , left_most:right_most]
            no_mask_mask = cv2.bitwise_and(area, area, mask=masks_mask)
            final_mask = cv2.add(no_mask_mask, mask)
            frame[0:mask.shape[0] , left_most:right_most] = final_mask
        else:
            mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            _, masks_mask = cv2.threshold(mask_gray, 25, 255, cv2.THRESH_BINARY_INV)
            area = frame[how_high - mask.shape[0]: how_high, left_most:right_most]
            no_mask_mask = cv2.bitwise_and(area, area, mask=masks_mask)
            final_mask = cv2.add(no_mask_mask, mask)
            frame[how_high - mask.shape[0]: how_high, left_most:right_most] = final_mask
        
        pts = []
        for pt in face_landmarks_list[0]['right_eye']:
            pts.append([[pt[0],pt[1]]])
        pts  = np.array(pts)
        cv2.polylines(frame,[pts],True,[0,0,255])
        cv2.fillPoly(frame,[pts],[0,0,255])
        
        pts = []
        for pt in face_landmarks_list[0]['left_eye']:
            pts.append([[pt[0],pt[1]]])
        pts  = np.array(pts)
        cv2.polylines(frame,[pts],True,[0,0,255])
        cv2.fillPoly(frame,[pts],[0,0,255])
    
    # Display the resulting frame
    cv2.imshow('frame',frame)
    #press q to exit loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
cap.release()
cv2.destroyAllWindows()